In [ ]:
!wget https://raw.githubusercontent.com/anmolsjoshi/nlptorch/master/layers/embedding.py

In [ ]:
!wget https://raw.githubusercontent.com/anmolsjoshi/nlptorch/master/layers/highway.py

In [ ]:
!wget https://raw.githubusercontent.com/anmolsjoshi/nlptorch/master/layers/attention.py

In [1]:
import torch as th
import torch.nn as nn

In [15]:
x = th.zeros([10, 600, 256])
y = th.zeros([256*3, 1])

In [ ]:
x_ = x.view(-1, x.shape[2])
x_.shape

In [18]:
cc = nn.Linear(768, 1).cuda()

In [ ]:
x.view(-1, x.shape[2]).shape

In [ ]:
cc(x_).shape

In [ ]:
cc(x_.transpose(1, 0)).shape

In [21]:
x = th.zeros([8, 600, 256])
y = th.zeros([8, 100, 256])

x = x.cuda()
y = y.cuda()

#x = x.unsqueeze(2).repeat(1, 1, 100, 1)
#y = y.unsqueeze(1).repeat(1, 600, 1, 1)

#p = th.mul(x, y)

In [32]:
I = th.bmm(x, y.transpose(1, 2))

In [33]:
I.shape

torch.Size([8, 600, 100])

In [19]:
w = cc(o)

In [20]:
w.shape

torch.Size([8, 600, 100, 1])

In [1]:
import torch as T
from torch import nn as nn
from torch.nn import functional as F

In [24]:
class CoAttention(nn.Module):
    def __init__(self, hidden_size):
        super(CoAttention, self).__init__()
        self.Wqj = nn.Linear(2*hidden_size, 2*hidden_size)
        self.c0 = nn.Parameter(T.rand(2*hidden_size,))
        self.q0 = nn.Parameter(T.rand(2 * hidden_size, ))
    
    def forward(self, q, c):
        b, _, l = q.shape
        qj = F.tanh(self.Wqj(q))
        q_vec = self.c0.unsqueeze(0).expand(b, l).unsqueeze(1)
        q = T.cat([qj, q_vec], dim=1)

        c_vec = self.c0.unsqueeze(0).expand(b, l).unsqueeze(1)
        c = T.cat([c, c_vec], dim=1)

        L = T.bmm(q, c.transpose(1, 2))
        alpha = F.softmax(L, dim=2)
        beta = F.softmax(L, dim=1)

        return alpha, beta

In [15]:
class BasicAttention(nn.Module):
    def __init__(self):
        super(BasicAttention, self).__init__()
    
    def forward(self, x, y):
        e = th.bmm(x, y.transpose(1, 2))
        ai = F.softmax(e, dim=-1)
        a = th.bmm(ai, y)
        b = th.cat([x, a], dim=-1)
        
        return b

In [25]:
x = T.zeros([8, 600, 256])
y = T.zeros([8, 100, 256])

x = x.cuda()
y = y.cuda()

In [26]:
i = CoAttention(128).cuda()
l, n = i(x, y)

In [27]:
l.shape

torch.Size([8, 601, 101])

In [28]:
n.shape

torch.Size([8, 601, 101])